### Loads

In [ ]:
!pip install yt-dlp
!pip install webvtt-py

In [ ]:
!pip install SpeechRecognition
!pip install git+https://github.com/openai/whisper.git soundfile

In [ ]:
!pip install transformers datasets evaluate jiwer

In [4]:
import webvtt

import speech_recognition as sr

from transformers import pipeline

import evaluate

Я долго не могла выбрать видео, в итоге я взяла 2 коротких, чтобы все модели сработали.

Первое видео -- это просто отрывок-скетч, это более интересный вариант. Тут говорит один человек, но как два разных персонажа, единственная проблема -- в конце много раз подряд произносится "ow", я не знала, как это стоит оценивать, потому что не все модели вообще распознают такие звуки. Но в итоге я решила сделать, как модель от OpenAI -- оставить эти звуки.

Второе видео -- это рассказ про обновление в игре с закадровым голосом, оно более скучное, так как здесь все четко произносится, поэтому у одной из моделей вообще получилось идеально распознать.

## Video

In [26]:
!yt-dlp -f ba --extract-audio --audio-format wav --write-auto-subs https://www.youtube.com/watch?v=EAfp8ZxQGpU #https://www.youtube.com/watch?v=0Cbw1b2mkGk

[youtube] Extracting URL: https://www.youtube.com/watch?v=0Cbw1b2mkGk#https://www.youtube.com/watch?v=EAfp8ZxQGpU
[youtube] 0Cbw1b2mkGk: Downloading webpage
[youtube] 0Cbw1b2mkGk: Downloading ios player API JSON
[youtube] 0Cbw1b2mkGk: Downloading android player API JSON
[youtube] 0Cbw1b2mkGk: Downloading m3u8 information
[info] 0Cbw1b2mkGk: Downloading subtitles: en
[info] 0Cbw1b2mkGk: Downloading 1 format(s): 251
[info] Writing video subtitles to: Minecraft 1.21： The breeze [0Cbw1b2mkGk].en.vtt
[download] Destination: Minecraft 1.21： The breeze [0Cbw1b2mkGk].en.vtt
[download] 100% of    8.52KiB in 00:00:00 at 313.99KiB/s
[download] Destination: Minecraft 1.21： The breeze [0Cbw1b2mkGk].webm
[download] 100% of    1.23MiB in 00:00:00 at 11.34MiB/s
[ExtractAudio] Destination: Minecraft 1.21： The breeze [0Cbw1b2mkGk].wav
Deleting original file Minecraft 1.21： The breeze [0Cbw1b2mkGk].webm (pass -k to keep)


In [54]:
subtitles_raw = []

for caption in webvtt.read("my favorite drew gooden clip [EAfp8ZxQGpU].en.vtt"): #"Minecraft 1.21： The breeze [0Cbw1b2mkGk].en.vtt"
    subtitles_raw.append(caption.text)

In [55]:
subtitles = []
for line in subtitles_raw:
    new_line = line.replace(' \n', '')
    new_line = new_line.replace('\n ', '')
    new_line = new_line.replace('\n', ' ')

    if new_line == ' ':
        continue
    else:
        subtitles.append(new_line)

In [56]:
subtitles[:20] # они не в особо удобном формате, я пробовала их почистить, но это вроде бы не нужно по заданию, так что я не стала особо этим заниматься

['definitely curious to try this out my',
 'definitely curious to try this out my',
 "definitely curious to try this out my only reservation is i don't know if",
 "only reservation is i don't know if",
 "only reservation is i don't know if we're allowed to install this on our",
 "we're allowed to install this on our",
 "we're allowed to install this on our toilets i'm gonna have to ask my",
 "toilets i'm gonna have to ask my",
 "toilets i'm gonna have to ask my landlord about it what's up",
 "landlord about it what's up",
 "landlord about it what's up were you just standing outside the door",
 'were you just standing outside the door',
 'were you just standing outside the door waiting for someone to mention you by',
 'waiting for someone to mention you by',
 'waiting for someone to mention you by name no i came here to tell you your',
 'name no i came here to tell you your',
 'name no i came here to tell you your dishwasher is broken well then fix it',
 'dishwasher is broken well then 

## SpeechRecognition

In [58]:
r = sr.Recognizer()
with sr.AudioFile("my favorite drew gooden clip [EAfp8ZxQGpU].wav") as source: #"Minecraft 1.21： The breeze [0Cbw1b2mkGk].wav"
    audio = r.record(source)

In [59]:
# OpenAI Whisper library
openai = r.recognize_whisper(audio)

In [60]:
openai

" I'm definitely curious to try this out. My only reservation is I don't know if we're allowed to install this on our toilets. I'm gonna have to ask my landlord about it. What's up? Were you just standing outside the door waiting for someone to mention you by name? No. I came here to tell you your dishwasher's broken. Well then fix it. That is not my job. That literally is your job. Also do you think my name is landlord? Yes. Landlord is not my name. It's my profession. My name is Ninja. My parents named me after the Fortnite streamer. How old are you? I gotta go. Oh man, this staircase is steep. Whoah. Ow. Ow. Ow. Ow. Ow. Ow. Oh. Oh. I'm okay. You know, I'm five days into this and I."

In [61]:
# Google Cloud Speech (with default key)
google = r.recognize_google(audio) # needs to be less than ~15 MB!!

In [62]:
google

"definitely curious to try this out my only reservation is I don't know if we're allowed to install this on our toilets I'm going to have to ask my landlord about it what's up were you just standing outside the door waiting for someone to mention you by name no I came here to tell you your dishwasher is broken well then fix it that is not my job that literally is your job also do you think my name is landlord landlord is not my name it's my professional my name is ninja my parents named me after the fortnite streamer how old are you I got to go this staircase is Steve you know"

## HuggingFace transformers

In [63]:
transcriber = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")
transformer = transcriber("my favorite drew gooden clip [EAfp8ZxQGpU].wav") # less than ~15 MB!! #"Minecraft 1.21： The breeze [0Cbw1b2mkGk].wav"

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [64]:
transformer

{'text': "EVELY CURIOUS TO TRY THIS OUT MY ONLY RESERVATION IS I DON'T KNOW IF WE'RE ALLOWED TO INSTALL THIS ON OUR TOILETS I'M GOING TO HAVE TO ASK MY LANDLORD ABOUT IT WHAT'S A WERE YOU JUST STANDING OUTSIDE THE DOOR WAITING FOR SOMEONE TO MENTION YOU BY NAME NO I CAME HERE TO TELL YOU YOUR DISHWATCHWARS BROKEN WOLVIN FIXET THAT IS NOT MY JA THAT LITERALLY IS YOUR JIAL SO DO YOU THINK MY NAME IS LANDLORD YES LANDLORD IS NOT MY NAME IT'S MY PROFESSION MY NAME IS NINJA MY PARENTS NAMEMIN AFTER THE FORTNIGHTS  HOW OLD ARE YOU AY A MA MISTERCASES SE WHO HOW OW OW HOW HOW HOW HW  YOU KNOW O FIVE DAYS IN ITUS IN A"}

## WER Video 1

In [21]:
wer = evaluate.load("wer")

In [65]:
references_1 = "Definitely curious to try this out. My only reservation is i don't know if we're allowed to install this on our toilets. I'm gonna have to ask my landlord about it. What's up? Were you just standing outside the door, waiting for someone to mention you by name? No. I came here to tell you your dishwasher is broken. Well then fix it. That is not my job. That literally is your job. Also do you think my name is landlord? Yes. Landlord is not my name it's my profession. My name is ninja. My parents named me after the fortnight streamer. How old are you? Uh i gotta go. Oh man this staircase is steep. Whoah. Ow. Ow. Ow. Ow. Ow. Ow. Oh. Oh. I'm okay. You know i'm five days into this and I."

In [66]:
elements = [',', '.', '?', '!'] # я убираю пунктуацию, потому что не все модели ее используют, за исключением апострофа, так как все модели различают it's и its, например (можно увидеть на примере второго видео)
for element in elements:
    references_1 = references_1.replace(element, '').lower()
references_1

"definitely curious to try this out my only reservation is i don't know if we're allowed to install this on our toilets i'm gonna have to ask my landlord about it what's up were you just standing outside the door waiting for someone to mention you by name no i came here to tell you your dishwasher is broken well then fix it that is not my job that literally is your job also do you think my name is landlord yes landlord is not my name it's my profession my name is ninja my parents named me after the fortnight streamer how old are you uh i gotta go oh man this staircase is steep whoah ow ow ow ow ow ow oh oh i'm okay you know i'm five days into this and i"

In [67]:
for element in elements:
    openai = openai.replace(element, '').lower()
openai

" i'm definitely curious to try this out my only reservation is i don't know if we're allowed to install this on our toilets i'm gonna have to ask my landlord about it what's up were you just standing outside the door waiting for someone to mention you by name no i came here to tell you your dishwasher's broken well then fix it that is not my job that literally is your job also do you think my name is landlord yes landlord is not my name it's my profession my name is ninja my parents named me after the fortnite streamer how old are you i gotta go oh man this staircase is steep whoah ow ow ow ow ow ow oh oh i'm okay you know i'm five days into this and i"

In [68]:
for element in elements:
    google = google.replace(element, '').lower()
google

"definitely curious to try this out my only reservation is i don't know if we're allowed to install this on our toilets i'm going to have to ask my landlord about it what's up were you just standing outside the door waiting for someone to mention you by name no i came here to tell you your dishwasher is broken well then fix it that is not my job that literally is your job also do you think my name is landlord landlord is not my name it's my professional my name is ninja my parents named me after the fortnite streamer how old are you i got to go this staircase is steve you know"

In [69]:
transformer = transformer['text']
for element in elements:
    transformer = transformer.replace(element, '').lower()
transformer

"evely curious to try this out my only reservation is i don't know if we're allowed to install this on our toilets i'm going to have to ask my landlord about it what's a were you just standing outside the door waiting for someone to mention you by name no i came here to tell you your dishwatchwars broken wolvin fixet that is not my ja that literally is your jial so do you think my name is landlord yes landlord is not my name it's my profession my name is ninja my parents namemin after the fortnights  how old are you ay a ma mistercases se who how ow ow how how how hw  you know o five days in itus in a"

In [70]:
wer_openai = wer.compute(predictions=[openai], references=[references_1])
wer_openai

0.03676470588235294

In [71]:
wer_google = wer.compute(predictions=[google], references=[references_1])
wer_google

0.21323529411764705

In [72]:
wer_transformer = wer.compute(predictions=[transformer], references=[references_1])
wer_transformer

0.3014705882352941

Лучше всех справился OpenAI. Можно заметить, что google не распознал те самые "ow". В целом самое большое расхождение в начале и в конце, потому что это отрывок видео, и начало и конец как раз обрезаны так, что можно не все распознать.

## WER Video 2

In [46]:
references_2 = "If you feel a gust of wind when exploring the new trial chambers, you might have stumbled upon a unique hostile mob, the Breeze. Despite its soothing name, the Breeze is quite a challenging addition to the game, and it's definitely not like any other mob you've fought before. It feels like it's angry, but it's also having fun at the same time, you know, like a cat, like playing around with its prey. The Breeze can jump in dizzying circles around its opponents and knock you back with powerful gusts of wind if you're not quick enough. It will hurt. When you meet the Breeze, the whole environment becomes more dangerous. Not to mention, you'll be fighting the Breeze in a room full of traps, and its bursts will not only send you flying, but can also trigger things like levers, buttons and more. So basically, anything can happen when you face a Breeze. It will hurt. Read more about how to try it out and send us your feedback on Minecraft.net."

In [47]:
elements = [',', '.', '?', '!']
for element in elements:
    references_2 = references_2.replace(element, '').lower()
references_2

"if you feel a gust of wind when exploring the new trial chambers you might have stumbled upon a unique hostile mob the breeze despite its soothing name the breeze is quite a challenging addition to the game and it's definitely not like any other mob you've fought before it feels like it's angry but it's also having fun at the same time you know like a cat like playing around with its prey the breeze can jump in dizzying circles around its opponents and knock you back with powerful gusts of wind if you're not quick enough it will hurt when you meet the breeze the whole environment becomes more dangerous not to mention you'll be fighting the breeze in a room full of traps and its bursts will not only send you flying but can also trigger things like levers buttons and more so basically anything can happen when you face a breeze it will hurt read more about how to try it out and send us your feedback on minecraftnet"

In [48]:
for element in elements:
    openai = openai.replace(element, '').lower()
openai

" if you feel a gust of wind when exploring the new trial chambers you might have stumbled upon a unique hostile mob the breeze despite its soothing name the breeze is quite a challenging addition to the game and it's definitely not like any other mob you've fought before it feels like it's angry but it's also having fun at the same time you know like a cat like playing around with its prey the breeze can jump in dizzying circles around its opponents and knock you back with powerful gusts of wind if you're not quick enough it will hurt when you meet the breeze the whole environment becomes more dangerous not to mention you'll be fighting the breeze in a room full of traps and its bursts will not only send you flying but can also trigger things like levers buttons and more so basically anything can happen when you face a breeze it will hurt read more about how to try it out and send us your feedback on minecraftnet"

In [49]:
for element in elements:
    google = google.replace(element, '').lower()
google

"if you feel a gust of wind when exploring the new trial chambers you might have stumbled upon a unique hostile mob the breeze despite its soothing name the breeze is quite a challenging addition to the game and it's definitely not like any other mop you for before it feels like it's angry but it's also having fun at the same time you know like a cat like playing around with its prey the breeze can jump circles around its opponents and knock you back with powerful gusts of wings if you are not enough he will hurt when you meet the breeze the whole entire become more dangerous in a room full of traps and it's bursts will not only send you things like levers buttons so basically anything can happen when you face a breeze it will hurt read more about how to try it out and send us your feedback on minecraftnet"

In [50]:
transformer = transformer['text']
for element in elements:
    transformer = transformer.replace(element, '').lower()
transformer

"if you feel a gust of wind when exploring the new trial chambers you might have stumbled upon a unique hostile mob the breeze despite its soothing name the breeze is quite a challenging addition to the game and it's definitely not like any other mot you fought before it feels like its angry but it's also having found at the same time you know like a cat like playing around with its pray the breeze can jump in dizzyig circles around its opponents and knock you back wit powerful gusts of wingd if you are not quick enough it will hurt when you meet the breeze the whole invironment becomes more dangerous not to mention yu be fining the breezes in a room full of traps and its bursts will not only send you flying but can also trigger things like levers an badons and new wort so basically anything can happen when you face a breeze it will hurt read more about how to try it out and send us your feeton mine craft do nets"

In [51]:
wer_openai = wer.compute(predictions=[openai], references=[references_2])
wer_openai

0.0

In [52]:
wer_google = wer.compute(predictions=[google], references=[references_2])
wer_google

0.16184971098265896

In [53]:
wer_transformer = wer.compute(predictions=[transformer], references=[references_2])
wer_transformer

0.1329479768786127

Опять же OpenAI справился лучше всех. Здесь интересно посмотреть, что только transformer не распознал, что "dot net" должно быть ".net" как часть сайта.